In [1]:
import numpy as np
import pandas as pd
import pyreadstat
import matplotlib.pyplot as plt

### Data variables required

### Data Info Key

EMSTotalCallTimeMin (COMPUTEDELEMENTS)
> Total dispatch to arrival hospital arrival time (main predictor): 

Einjury.3: (factpcrtraumacriteria)
> trauma triage criteria high risk
Binarize: yes (has some value here) or no 

Einjury4: trauma triage criteria moderate risk (factpcrinjuryriskfactor)
> Binarize: yes (has some value here) or no 
At this point, patients should either have a 1 0, 0 1, or 0 0 for einjury3/4, corresponding to high risk, moderate risk, low risk 
Can create new column "trauma triage criteria: high vs moderate vs low 

EDisposition.23 - hospital capability (PCREVENTS)
>delineate whether TC level 1-5, or other (tabulate)

EPatient.13: gender (PUB_PCREVENTS)

EVitals.06: SPB (FACTPCRVITAL)

Evitals.10: HR (FACTPCRVITAL)

Evitals. 19-22: GCS (FACTPCRVITAL)

EMedications.03-medications administered (FACTPCRMEDICATION)
> Look at "allowable snomed codes: and classify whether any  blood product (blood product, cryo, ffp, platelets, whole blood, packed rbc-- NOT albumin)

Eoutcome.01: ED disposition (pub_pcrevents)
> Delineate expired vs survived (discharged to other places like home)
we may eventually need to think about delineating  ED death vs those who survived ED but died in hospital (e.g. Did these patients die because transport time was too long vs they were non survival injuries and were already destined to die before arrival...). Could you tabulate how many died in ED? 

Eoutcome.02: hospital disposition (pub_pcrevents)
> Delineate expired vs survived (discharged to other places like home)
I believe this is how we get our outcome of hospital survival  

### Import Inclusion + Exclusion Data

In [2]:
trauma_df = pd.read_csv('/Volumes/Research/GoldenHourData/InclusionCriteria/InclusionExclusionCriteria.csv')


### Merge Morality and Transportation Data

In [ ]:
# mortality data
pcrevents_df, _ = pyreadstat.read_sas7bdat('/Volumes/Research/GoldenHourData/NEMSISRawFiles/pub_pcrevents.sas7bdat', 
                                           usecols=["PcrKey", "eOutcome_01", "eOutcome_02"])

# merge mortality data
trauma_m_df = trauma_df.merge(pcrevents_df, on='PcrKey', how='left', validate='1:1')

In [ ]:
# extract other desired properties of incidents
CE_df, _ = pyreadstat.read_sas7bdat('/Volumes/Research/GoldenHourData/NEMSISRawFiles/computedelements.sas7bdat', 
                                    usecols=['PcrKey', 'EMSTotalCallTimeMin'])

# merge transportation data
trauma_mt_df = trauma_m_df.merge(CE_df, on='PcrKey', how='left', validate='1:1')

In [ ]:
# remove null transportation time
null_transport_idx = trauma_mt_df['EMSTotalCallTimeMin'].isnull()
print(f"# Null transport: {null_transport_idx.sum()}")
trauma_mt_df_denullT_df = trauma_mt_df[~null_transport_idx]


In [ ]:
# remove all "Not applicable" or "Not recorded" outcomes
# eOutcome_01 - ED disposition, eOutcome_02 - hospital disposition
ed_outcome_null_idx = (trauma_mt_df_denullT_df['eOutcome_01']== '7701001') | (trauma_mt_df_denullT_df['eOutcome_01'] == '7701003')
hospital_outcome_null_idx = (trauma_mt_df_denullT_df['eOutcome_02']== '7701001') | (trauma_mt_df_denullT_df['eOutcome_02'] == '7701003')

no_outcome_idx = ed_outcome_null_idx & hospital_outcome_null_idx
trauma_mt_denullTO_df = trauma_mt_df_denullT_df[~no_outcome_idx]

null_idx = trauma_mt_df_denullT_df['eOutcome_01'].isnull() & trauma_mt_df_denullT_df['eOutcome_02'].isnull()

print(f"# Null mortality info: {null_idx.sum()}")
print(f"# No mortality info: {no_outcome_idx.sum()}")
print(f"# mortality info cases: {len(trauma_mt_denullTO_df)}")


In [ ]:
# number of deceased or expired from injury
deceased_idx = (trauma_mt_denullTO_df['eOutcome_01'] == '20') | (trauma_mt_denullTO_df['eOutcome_02'] == '20')
print(f"Number of deceased/expired trauma cases: {np.sum(deceased_idx)}")
print(f"Number of alive trauma cases: {np.sum(~deceased_idx)}")

trauma_mt_denullTO_df['death'] = False
trauma_mt_denullTO_df.loc[deceased_idx, 'death'] = True


In [ ]:
trauma_mt_denullTO_df.to_csv("/Volumes/Research/GoldenHourData/Mortality/Mort_Trans_trauma.csv", index=False)

### Incorporate other dependent variables

- eDisposition.23 - Hospital Capability (PUB_PCREVENTS)
- ePatient.13 - Gender (PUB_PCREVENTS)
- ePatient.15 - Age (PUB_PCREVENTS)
- ePatient.14 - Race (PCRPATIENTRACEGROUP)

In [ ]:
trauma_mt_denullTO_df = pd.read_csv("/Volumes/Research/GoldenHourData/Mortality/Mort_Trans_trauma.csv")

In [ ]:
pcrevents_df, _ = pyreadstat.read_sas7bdat('/Volumes/Research/GoldenHourData/NEMSISRawFiles/pub_pcrevents.sas7bdat', 
                                           usecols=["PcrKey", "eDisposition_23", "ePatient_13"])

In [ ]:
# merge hospital capability, gender age
trauma_HGA_df = trauma_mt_denullTO_df.merge(pcrevents_df, on='PcrKey', how='left', validate='1:1')

In [ ]:
# convert gender
mapping_dict = {
    "9906001":'Female',
    "9906003":'Male'
}

trauma_HGA_df['ePatient_13_c'] = trauma_HGA_df['ePatient_13'].map(mapping_dict)

trauma_HGA_df.loc[trauma_HGA_df['ePatient_13_c'] == '9906005', 'ePatient_13_c'] = np.nan


In [ ]:
trauma_HGA_df['ePatient_13_c'].value_counts()

In [ ]:
trauma_HGA_df['ePatient_13_c'].isnull().sum()

In [ ]:
# convert hospital capability
mapping_dict = {
    "9908001" :'Behavioral Health',
    "9908003" :'Burn Center',
    "9908005" :'Critical Access Hospital',
    "9908007" :'Hospital (General)',
    "9908009" :'Neonatal Center',
    "9908011" :'Pediatric Center',
    "9908017" :'Stroke Center',
    "9908019" :'Rehab Center',
    '9908021' :'Trauma Center Level 1',
    '9908023' :'Trauma Center Level 2',
    '9908025' :'Trauma Center Level 3',
    '9908027' :'Trauma Center Level 4',
    '9908029' :'Trauma Center Level 5',
    '9908031' :'Cardiac-STEMI/PCI Capable',
    '9908033' :'Cardiac-STEMI/PCI Capable (24/7)',
    '9908035' :'Cardiac-STEMI/Non-PCI Capable'
}

trauma_HGA_df['eDisposition_23_c'] = trauma_HGA_df['eDisposition_23'].map(mapping_dict)

trauma_HGA_df.loc[trauma_HGA_df['eDisposition_23'] == '7701001', 'eDisposition_23_c'] = np.nan
trauma_HGA_df.loc[trauma_HGA_df['eDisposition_23'] == '7701003', 'eDisposition_23_c'] = np.nan

In [ ]:
trauma_HGA_df['eDisposition_23_c'].value_counts()

In [ ]:
# organize disposition site based on the following cateogires
# categorical variable. classify  as level 1 or 2 vs level 3 to 5 vs not trauma center 

mapping_dict = {
    "9908001" :'not trauma center',
    "9908003" :'not trauma center',
    "9908005" :'not trauma center',
    "9908007" :'not trauma center',
    "9908009" :'not trauma center',
    "9908011" :'not trauma center',
    "9908017" :'not trauma center',
    "9908019" :'not trauma center',
    '9908021' :'Trauma Center Level 1-2',
    '9908023' :'Trauma Center Level 1-2',
    '9908025' :'Trauma Center Level 3-5',
    '9908027' :'Trauma Center Level 3-5',
    '9908029' :'Trauma Center Level 3-5',
    '9908031' :'not trauma center',
    '9908033' :'not trauma center',
    '9908035' :'not trauma center'
}

trauma_HGA_df['eDisposition_23_c_group'] = trauma_HGA_df['eDisposition_23'].map(mapping_dict)

trauma_HGA_df.loc[trauma_HGA_df['eDisposition_23'] == '7701001', 'eDisposition_23_c_group'] = np.nan
trauma_HGA_df.loc[trauma_HGA_df['eDisposition_23'] == '7701003', 'eDisposition_23_c_group'] = np.nan

In [ ]:
trauma_HGA_df['eDisposition_23_c_group'].value_counts()

In [ ]:
trauma_HGA_df['eDisposition_23_c_group'].isnull().sum()

In [ ]:
trauma_HGA_df.to_csv("Data/Mortality/Mort_Trans_trauma_FINAL.csv", index=False)

### Format data for R RDD

- age
- travel_time
- RTS

In [ ]:
trauma_HGA_df = pd.read_csv("Data/Mortality/Mort_Trans_trauma_FINAL.csv")

In [ ]:
keep_cols = ['EMSTotalCallTimeMin', 'ePatient_15','RTS', 'eDisposition_23_c_group', 'ePatient_13_c', 'death']
trauma_HGA_filt_df = trauma_HGA_df[keep_cols]

In [ ]:
trauma_HGA_filt_df.sample(2)

In [ ]:
trauma_HGA_filt_df = trauma_HGA_filt_df.rename(columns={"EMSTotalCallTimeMin": "travel_time", 
                                   "ePatient_15": "age",
                                  "RTS": "RTS", 
                                   "eDisposition_23_c_group": "trauma_center",
                                  "ePatient_13_c": "gender", 
                                   "death": "mortality"})


In [ ]:
trauma_HGA_filt_df.sample(2)

In [ ]:
trauma_HGA_filt_df.to_csv("Data/Mortality/Mort_Trans_trauma_FINAL_RDDforR.csv", index=False)

### Assess Number of Mortality Cases

https://journals.plos.org/plosone/article?id=10.1371/journal.pone.0276755

https://pmc.ncbi.nlm.nih.gov/articles/PMC7001471/

- May need to take predictive modeling approach (rather than inference modeling). The function has a main predictor of transport time (continuous variable) and other confounders and effect modifiers (effect modifiers: I think would be ISS, GCS, perhaps blunt vs penetrating moi), outcome being hospital mortality
- Use regression discontinuity to find optimal threshold at which mortality is minimized (may need to inverse label as survival to maximize outcome) 
- Apply threshold, using Bayesian methods, on a separate dataset (e.g. Insert model that has same covariate values otherwise but we change the transport time, to see how many potential more raw # of lives we can have saved and how many yeears of potential life lost we could have saved <- this will be the kicker 

Check out AAST 2024 podium paper abstracts (#1-6 are the "top" abstracts)
https://www.aast.org/annual-meeting/program

### Graph RTS with transportation time

In [ ]:
trauma_mt_denullTO_df

In [ ]:
death_idx = trauma_mt_denullTO_df['death']
# colors = np.where(death_idx, 'red', 'blue')


plt.scatter(trauma_mt_denullTO_df.loc[~death_idx, 'EMSTotalCallTimeMin'], trauma_mt_denullTO_df.loc[~death_idx, 'RTS'],
           alpha=0.07, c='blue')
plt.scatter(trauma_mt_denullTO_df.loc[death_idx, 'EMSTotalCallTimeMin'], trauma_mt_denullTO_df.loc[death_idx, 'RTS'],
           alpha=0.2, c='red')
plt.ylabel("RTS"); plt.xlabel("Transportation Time (Min)")
plt.show()

### Make bins for percent mortality

In [ ]:
max_time = trauma_mt_denullTO_df['EMSTotalCallTimeMin'].max()

In [ ]:
trans_time = trauma_mt_denullTO_df['EMSTotalCallTimeMin']

bin_size = 10
bin_limits = np.arange(0,max_time,5)
perc_mort_list = []

for lower_lim in bin_limits:
    upper_lim = lower_lim + bin_size
    cond = (trans_time >= lower_lim) & (trans_time < upper_lim)
    num_death = trauma_mt_denullTO_df.loc[cond, 'death'].sum()
    perc_mort = 100 * num_death / len(trauma_mt_denullTO_df)
    perc_mort_list.append(perc_mort)

perc_mort_list = np.array(perc_mort_list)

In [ ]:
fig, ax = plt.subplots(figsize=(7,5))
plt.bar(bin_limits, perc_mort_list, width=bin_size/2)
plt.xlim([0,200])
plt.ylabel('Percent Mortality (%)'); plt.xlabel("Transportation time (min)")
plt.title("Trauma Mortality with Transportation Time")
plt.axvline(60, c='r')
plt.show()


### Logistic Regression Model Statistical Test

In [ ]:
# trauma_HGA_df = pd.read_csv("Data/Mortality/Mort_Trans_trauma_FINAL.csv")
trauma_HGA_df = pd.read_csv("Data/Mortality/Mort_Trans_trauma_FINAL_RDDforR.csv")

In [ ]:
trauma_HGA_df.sample(2)

In [ ]:
trauma_HGA_df['mortaltiy_c'] = 0
trauma_HGA_df.loc[trauma_HGA_df['mortality'], 'mortaltiy_c'] = 1

In [ ]:
dependent_var = ['mortaltiy_c']
cont_independent_var = ['travel_time', 'RTS', 'age']

cat_independent_var = ['trauma_center', 'gender']
model_data = trauma_HGA_df[dependent_var+cont_independent_var+cat_independent_var]

model_data = model_data.dropna()
print(f"% TCs remaining: {100*len(model_data)/len(trauma_HGA_df)}")

In [ ]:
model_data['trauma_center'].value_counts()

In [ ]:
import statsmodels.formula.api as smf

formula = dependent_var[0] + " ~ " + " + ".join(cont_independent_var) + " + " + " + ".join([f"C({x})" for x in cat_independent_var])
formula = formula.replace(" C(trauma_center)"," C(trauma_center, Treatment(reference='not trauma center'))")
formula


In [ ]:
model = smf.logit(formula, data=model_data)
res = model.fit()


In [ ]:
print(res.summary())

In [ ]:
data = {'coef': model_fit_regularized.params,
        'std err': model_fit_regularized.bse,
        'P>|t|': model_fit_regularized.pvalues,
        '[0.025': model_fit_regularized.conf_int()[0],
        '0.975]': model_fit_regularized.conf_int()[1]}

result = pd.DataFrame(data)

In [ ]:
result

### Logistic Regression


In [ ]:
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import confusion_matrix, roc_curve, roc_auc_score
import seaborn as sns

In [ ]:
X = trauma_mt_denullTO_df['EMSTotalCallTimeMin'].to_numpy().reshape(-1, 1)
y = trauma_mt_denullTO_df['death'].to_numpy()

X_train, X_test, y_train, y_test = train_test_split(X, y, 
                                                    test_size=0.33, random_state=42)



In [ ]:
# Train logistic regression model
model = LogisticRegression()
model.fit(X_train, y_train)

# Predict probabilities on the test set
y_prob = model.predict_proba(X_test)[:, 1]
# Predict classes on the test set
y_pred = model.predict(X_test)

In [ ]:
# Generate points for the prediction line
X_gen = np.linspace(0, 200, 300)[:, np.newaxis]
y_gen = model.predict_proba(X_gen)[:, 1]  # Probability of mortality

# Plot data points
plt.scatter(X_train, y_train, color='black', label="Data points", s=3, alpha=0.3)

# Plot logistic curve
plt.plot(X_gen, y_gen, color='blue', linewidth=2, label="Logistic curve")

# Labels and title
plt.xlabel('Transportation Time')
plt.ylabel('Mortality Probability')
plt.title('Logistic Regression of Mortality on Transportation Time')
plt.xlim([0,200])
plt.legend()
plt.show()

In [ ]:
# Confusion matrix
cm = confusion_matrix(y_test, y_pred)
print("Confusion Matrix:")
print(cm)

In [ ]:
# Plot confusion matrix heatmap
plt.figure(figsize=(8, 6))
sns.heatmap(cm, annot=True, fmt="d", cmap="Blues", cbar=False,
            xticklabels=["Predicted Negative", "Predicted Positive"],
            yticklabels=["Actual Negative", "Actual Positive"])
plt.title("Confusion Matrix")
plt.xlabel("Predicted")
plt.ylabel("Actual")
plt.show()

In [ ]:
# ROC curve
fpr, tpr, thresholds = roc_curve(y_test, y_prob)

# AUC score
roc_auc = roc_auc_score(y_test, y_prob)
print(f"AUC: {roc_auc}")

# Plot ROC curve
plt.figure(figsize=(8, 6))
plt.plot(fpr, tpr, color='darkorange', lw=2, label=f'ROC curve (area = {roc_auc:.2f})')
plt.plot([0, 1], [0, 1], color='navy', lw=2, linestyle='--')
plt.xlim([0.0, 1.0])
plt.ylim([0.0, 1.05])
plt.xlabel('False Positive Rate')
plt.ylabel('True Positive Rate')
plt.title('Receiver Operating Characteristic (ROC)')
plt.legend(loc="lower right")
plt.show()